In [ ]:
import numpy as np
from numpy import linalg as LA
import cv2 as cv
import math

In [ ]:
def fm_estimation(f1,f2, nums):

  #nums is the indices of points , f1 contains all the points while f2 contains the corresponding points of f1

  A = []
  f1 = np.array(f1 , dtype = 'float32')
  f2 = np.array(f2 , dtype = 'float32')

  for iter in range(8):

    x = f1[nums[iter]][0]
    x_ = f2[nums[iter]][0]
    y = f1[nums[iter]][1]
    y_ = f2[nums[iter]][1]

    temp_row = [x*x_ , x*y_, x , y*x_ ,y*y_ , y , x_ , y_, 1 ]
    A.append(temp_row)


  A = np.array(A)

  A_t = A.T

  M = np.matmul(A_t , A)
  u, s, vh = np.linalg.svd(M, full_matrices=False)
  mc_index = np.argmin(s)
  f = vh[mc_index].reshape((3,3))

  return f


In [ ]:
def normalize_points(c1, num):

  #num is the total number of points in the dataset

  sum1 = 0
  sum2 = 0
  result = []

  for i in range(num):

    sum1+= c1[i][0]
    sum2+= c1[i][1]

  cx1 = sum1/num
  cy1 = sum2/num


  sx1 = np.var(c1 , axis = 0)[0]
  sy1 = np.var(c1 , axis = 0)[1]

  T1 = np.array([[sx1 , 0 ,0] , [0 , sy1 , 0] , [0,0,1]])
  T2 = np.array([[1 , 0 , -cx1] , [0 , 1 , -cy1] , [0 ,0 ,1]])

  for i in range(num):

    temp = np.matmul(T1 , np.matmul(T2 , np.hstack((c1[i] , 1)).T))
    temp = [temp[0] , temp[1]]
    result.append(temp)

  result = np.array(result)
  return result

In [ ]:
pic1 = [[880 , 214],
 [43 , 203],
[270 , 197],
[886  ,347],
[745 , 302],
[943 , 128],
[476 , 590],
[419 , 214],
[317 , 335],
[783 , 521],
[235 , 427],
[665 , 429],
[655 , 362],
[427 , 333],
[412  ,415],
[746  ,351],
[434 , 415],
[525 , 234],
[716 , 308],
[602 , 187]]

pic1 = np.array(pic1 , dtype = 'float32')

pic2 =[[ 731,  238],
[22   ,248],
[204  ,230],
[903  ,342],
[635  ,316],
[867  ,177],
[958  ,572],
[328  ,244],
[426  ,386],
[1064 ,470],
[480  ,495],
[964  ,419],
[695  ,374],
[505  ,372],
[645  ,452],
[692  ,359],
[712  ,444],
[465  ,263],
[591  ,324],
[447  ,213]]



# RanSAC

In [ ]:
from numpy.random.mtrand import random_integers
import random

def RANSAC_8 (p, p_):

  # p denotes the set of points of one image and p_ is the set of points corresponding to p in the other image

  max_inliers = 0
  inliers_list = []
  final_inliers_list = []

  # p = normalize_points(p , len(p))
  # p_ = normalize_points(p_ , len(p_))

  for it in range(1000): # number of times to run and seek better values of F

    inliers = 0

    # get 8 points at random
    indices = random.sample(range(len(p)), 8)

    Fm = fm_estimation(p, p_, indices)

    for i in range(len(p)):

      # epipolar contraint eqn
      Cstr = abs(np.matmul(np.hstack((p_[i], 1)), np.matmul(Fm, np.hstack((p[i], 1)).T)))

      if Cstr < 0.05:
        inliers += 1
        inliers_list.append(i)

    if inliers >= max_inliers:
      max_inliers = inliers
      bestFm = Fm
      final_inliers_list = inliers_list

    inliers_list = []

  return bestFm , final_inliers_list


In [ ]:
ans1 = RANSAC_8(pic1, pic2)
ans1

(array([[-3.60740019e-08,  5.42122073e-06, -1.80353276e-03],
        [-3.11175950e-06,  6.19494873e-06, -1.35554929e-03],
        [ 1.23477655e-03, -4.15396729e-03,  9.99988065e-01]]),
 [0, 3, 4, 5, 10, 12, 14, 15, 17, 18])